In [1]:
!pip install wandb

In [2]:
import os
from transformers import AutoTokenizer,AutoModelForCausalLM

In [ ]:
os.environ["HF_TOKEN"]=os.getenv("hf_api_key")
os.environ["WANDB_DISABLED"] = "true"

In [4]:
hf_model="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer=AutoTokenizer.from_pretrained(hf_model)
model=AutoModelForCausalLM.from_pretrained(hf_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig,get_peft_model,TaskType

In [6]:
configuration=LoraConfig(task_type=TaskType.CAUSAL_LM,
                         r=4,
                         lora_alpha=8,
                         lora_dropout=0.1,
                         bias="none",
                         target_modules=["q_proj", "k_proj", "v_proj", "o_proj"])


lora_model=get_peft_model(model=model,peft_config=configuration)

In [7]:
lora_model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [8]:
with open("Q&A outputs.txt","r") as fi:
  lk=[]
  for l in fi:
    lk.append(l)

In [9]:
# lk

In [10]:
import re

def load_qa_pairs(file_name):
  pairs=[]

  with open(file_name,"r") as f:
    lines=[]

    for l in f:
      if 'here' in l.strip().lower():
        continue
      if l.strip() and l.strip()!='\n':
        l=l.strip().replace('\n','')
        lines.append(l)

    for i in range(0,len(lines),2):
      if i+1 < len(lines):
        q=re.sub(r'^Q\d+:\s*', '', lines[i]).strip()
        a=re.sub(r'^A\d+:\s*', '', lines[i+1]).strip()
        pairs.append({"question":q,"answer":a})

  return pairs

In [11]:
q_a_data=load_qa_pairs("Q&A outputs.txt")

In [12]:
# q_a_data

In [13]:
!pip install datasets

In [14]:
from datasets import Dataset
q_a_data=Dataset.from_list(q_a_data)
q_a_data

Dataset({
    features: ['question', 'answer'],
    num_rows: 280
})

In [15]:
def tokenize_function(example):
    tokenized = tokenizer(
        example['question'],
        padding="max_length",
        max_length=512,
        truncation=True
    )

    labels_generated = tokenizer(
        example['answer'],
        padding="max_length",
        max_length=512,
        truncation=True
    )

    labels = labels_generated['input_ids']
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]

    tokenized['labels'] = labels

    return tokenized


In [16]:
tokenized_q_a_data=q_a_data.map(tokenize_function,remove_columns=['question','answer'])

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

In [17]:
tokenized_q_a_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 280
})

In [18]:
tokenizer.pad_token_id

2

In [19]:
print(tokenized_q_a_data[0])

{'input_ids': [1, 1724, 526, 777, 18853, 2299, 1169, 363, 7344, 292, 1781, 9045, 29973, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [20]:
import torch

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [22]:
from transformers import Trainer,TrainingArguments

In [23]:
training_arguments=TrainingArguments(output_dir=r"./llama3-lora-qa",
                                     per_device_train_batch_size=1,
                                     gradient_accumulation_steps=4,
                                     learning_rate=0.001,
                                     num_train_epochs=3,
                                     logging_steps=10,
                                     save_strategy="epoch",
                                     fp16=False,
                                     label_names=["labels"])

# training_arguments

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
trainer=Trainer(model=lora_model,
        args=training_arguments,
        train_dataset=tokenized_q_a_data,
        # tokenizer=tokenizer,
        data_collator=data_collator
        )

In [25]:
trainer.train()

Step,Training Loss
10,2.255300
20,1.858200
30,1.924400
40,1.910600
50,1.961400
60,1.739100
70,1.867100
80,1.584100
90,1.404300
100,1.612200


TrainOutput(global_step=210, training_loss=1.544112069266183, metrics={'train_runtime': 545.831, 'train_samples_per_second': 1.539, 'train_steps_per_second': 0.385, 'total_flos': 2672445169336320.0, 'train_loss': 1.544112069266183, 'epoch': 3.0})

In [26]:
lora_model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [27]:
print(type(lora_model))

<class 'peft.peft_model.PeftModelForCausalLM'>


In [28]:
lora_model.save_pretrained("./llama3-lora-qa")
tokenizer.save_pretrained("./llama3-lora-qa")

('./llama3-lora-qa/tokenizer_config.json',
 './llama3-lora-qa/special_tokens_map.json',
 './llama3-lora-qa/chat_template.jinja',
 './llama3-lora-qa/tokenizer.model',
 './llama3-lora-qa/added_tokens.json',
 './llama3-lora-qa/tokenizer.json')

In [29]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(hf_model)
model = PeftModel.from_pretrained(base_model, "./llama3-lora-qa")

tokenizer = AutoTokenizer.from_pretrained("./llama3-lora-qa")
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
  

In [54]:
question = "what leads to hear attacks?"

inputs = tokenizer(question, return_tensors="pt").to("cpu")

with torch.no_grad():
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )

response = tokenizer.decode(output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)


In [55]:
response

'what leads to hear attacks?\n\nthe common cause of auditory neuropathy, also known as auditory neuropathy syndrome, is inner ear disease, particularly meningitis or syringobacterial meningitis. Women are more likely to develop this condition. The onset of symptoms can be sudden, starting with an auditory hallucination or high-pitched sounds, followed by a sudden loss of hearing. The syndrome can progress to complete deafness. Tre'

In [56]:
import shutil
import zipfile

shutil.make_archive("llama3-lora-qa", 'zip', "llama3-lora-qa")

'/content/llama3-lora-qa.zip'